In [37]:
import requests
import pandas as pd
from datetime import date, timedelta
import numpy as np

# Data Retrieving

## Retrieve flights data

In [30]:
start_date = date(2021,12,1)   # start date
end_date = date(2022,3,1) 

date_array = pandas.date_range(start_date,end_date-timedelta(days=1),freq='d')#.tolist()
date_array = date_array.strftime('%Y-%m-%d').tolist()
print(date_array)

['2021-12-01', '2021-12-02', '2021-12-03', '2021-12-04', '2021-12-05', '2021-12-06', '2021-12-07', '2021-12-08', '2021-12-09', '2021-12-10', '2021-12-11', '2021-12-12', '2021-12-13', '2021-12-14', '2021-12-15', '2021-12-16', '2021-12-17', '2021-12-18', '2021-12-19', '2021-12-20', '2021-12-21', '2021-12-22', '2021-12-23', '2021-12-24', '2021-12-25', '2021-12-26', '2021-12-27', '2021-12-28', '2021-12-29', '2021-12-30', '2021-12-31', '2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08', '2022-01-09', '2022-01-10', '2022-01-11', '2022-01-12', '2022-01-13', '2022-01-14', '2022-01-15', '2022-01-16', '2022-01-17', '2022-01-18', '2022-01-19', '2022-01-20', '2022-01-21', '2022-01-22', '2022-01-23', '2022-01-24', '2022-01-25', '2022-01-26', '2022-01-27', '2022-01-28', '2022-01-29', '2022-01-30', '2022-01-31', '2022-02-01', '2022-02-02', '2022-02-03', '2022-02-04', '2022-02-05', '2022-02-06', '2022-02-07', '2022-02-08', '2022-02-09', '2022

In [28]:
max_offset=1001
data = []
for a_date in date_array:
    for cur_offset in range(0,max_offset,100):
        params = {
          'access_key': '9157b4a71cda79091103bc987999f47e',
          'flight_date': a_date
        }
        api_result = requests.get('https://api.aviationstack.com/v1/flights?offset=' + str(cur_offset), params)
        api_response = api_result.json()['data']
        part_df = pd.json_normalize(api_response)
        data.append(part_df)
    #print('{}%'.format(100*cur_offset/(max_offset-1)))
    
df = pd.concat(data)
df.to_csv('./data/winter_2021.csv',index=False)

## Retrive airports data

In [32]:
max_offset = 6401
airport_data = []
for cur_offset in range(0,max_offset,100):
    params = {
      'access_key': '9157b4a71cda79091103bc987999f47e',
    }
    api_result = requests.get('https://api.aviationstack.com/v1/airports?offset=' + str(cur_offset), params)
    api_response = api_result.json()['data']
    part_df = pd.json_normalize(api_response)
    airport_data.append(part_df)
df = pd.concat(airport_data)
df.to_csv('./data/airport.csv',index=False)

## Retrive airlines data

In [33]:
max_offset = 13101
airline_data = []
for cur_offset in range(0,max_offset,100):
    params = {
      'access_key': '9157b4a71cda79091103bc987999f47e',
    }
    api_result = requests.get('https://api.aviationstack.com/v1/airlines?offset=' + str(cur_offset), params)
    api_response = api_result.json()['data']
    part_df = pd.json_normalize(api_response)
    airline_data.append(part_df)
df = pd.concat(airline_data)
df.to_csv('./data/airline.csv',index=False)

# Data Cleaning

## Clean flights_data

#### 1.remove columns that has more than 40% missing data

In [127]:
flights_df = pd.read_csv('./data/winter_2021.csv')
print('Before:',flights_df.columns.tolist())
flights_df  = flights_df.replace([0,' ','NULL'],np.nan)
flights_df = flights_df.dropna(axis=1,thresh=len(flights_df)*0.60)
print('')
print('After:',flights_df.columns.tolist())

Before: ['flight_date', 'flight_status', 'live', 'departure_airport', 'departure_timezone', 'departure_iata', 'departure_icao', 'departure_terminal', 'departure_gate', 'departure_delay', 'departure_scheduled', 'departure_estimated', 'departure_actual', 'departure_estimated_runway', 'departure_actual_runway', 'arrival_airport', 'arrival_timezone', 'arrival_iata', 'arrival_icao', 'arrival_terminal', 'arrival_gate', 'arrival_baggage', 'arrival_delay', 'arrival_scheduled', 'arrival_estimated', 'arrival_actual', 'arrival_estimated_runway', 'arrival_actual_runway', 'airline_name', 'airline_iata', 'airline_icao', 'flight_number', 'flight_iata', 'flight_icao', 'flight_codeshared', 'aircraft_registration', 'aircraft_iata', 'aircraft_icao', 'aircraft_icao24', 'live_updated', 'live_latitude', 'live_longitude', 'live_altitude', 'live_direction', 'live_speed_horizontal', 'live_speed_vertical', 'live_is_ground', 'aircraft', 'flight_codeshared_airline_name', 'flight_codeshared_airline_iata', 'flight_

#### 2. remove rows that don't have departure_icao, departure_scheduled/estimated
#### arrival_icao, arrival_scheduled/estimated 

In [128]:
print('# rows before:',len(flights_df))
flights_df = flights_df[flights_df['departure_icao'].notna()]
flights_df = flights_df[flights_df['departure_scheduled'].notna()]
flights_df = flights_df[flights_df['arrival_scheduled'].notna()]
flights_df = flights_df[flights_df['departure_estimated'].notna()]
flights_df = flights_df[flights_df['arrival_icao'].notna()]
flights_df = flights_df[flights_df['arrival_estimated'].notna()]
print('# rows after:',len(flights_df))

# rows before: 93500
# rows after: 93236


#### 3.  Change datetime columns from string to datetime.

In [129]:
time_columns = ['flight_date','departure_scheduled', 'departure_estimated', 'departure_actual',
                'departure_estimated_runway', 'departure_actual_runway','arrival_scheduled', 
                'arrival_estimated', 'arrival_actual', 'arrival_estimated_runway', 'arrival_actual_runway']
for col in time_columns:
    flights_df[col] = pd.to_datetime(flights_df[col])
flights_df

,flight_date,flight_status,departure_airport,departure_timezone,departure_iata,departure_icao,departure_gate,departure_delay,departure_scheduled,departure_estimated,...,arrival_estimated,arrival_actual,arrival_estimated_runway,arrival_actual_runway,airline_name,airline_iata,airline_icao,flight_number,flight_iata,flight_icao
0,2021-12-06,landed,John F Kennedy International,America/New_York,JFK,KJFK,35,35.0,2021-12-06 16:35:00+00:00,2021-12-06 16:35:00+00:00,...,2021-12-06 19:55:00+00:00,2021-12-06 20:27:00+00:00,2021-12-06 20:27:00+00:00,2021-12-06 20:27:00+00:00,American Airlines,AA,AAL,429.0,AA429,AAL429
1,2021-12-06,landed,John F Kennedy International,America/New_York,JFK,KJFK,29,52.0,2021-12-06 16:30:00+00:00,2021-12-06 16:30:00+00:00,...,2021-12-06 19:37:00+00:00,2021-12-06 19:52:00+00:00,2021-12-06 19:52:00+00:00,2021-12-06 19:52:00+00:00,JetBlue Airways,B6,JBU,1401.0,B61401,JBU1401
2,2021-12-06,landed,John F Kennedy International,America/New_York,JFK,KJFK,17,34.0,2021-12-06 16:30:00+00:00,2021-12-06 16:30:00+00:00,...,2021-12-06 19:50:00+00:00,2021-12-06 19:53:00+00:00,2021-12-06 19:53:00+00:00,2021-12-06 19:53:00+00:00,JetBlue Airways,B6,JBU,263.0,B6263,JBU263
3,2021-12-06,landed,John F Kennedy International,America/New_York,JFK,KJFK,2,56.0,2021-12-06 16:30:00+00:00,2021-12-06 16:30:00+00:00,...,2021-12-07 05:50:00+00:00,2021-12-07 05:30:00+00:00,2021-12-07 05:30:00+00:00,2021-12-07 05:30:00+00:00,Air France,AF,AFR,23.0,AF23,AFR23
4,2021-12-06,landed,John F Kennedy International,America/New_York,JFK,KJFK,6,35.0,2021-12-06 16:30:00+00:00,2021-12-06 16:30:00+00:00,...,2021-12-06 20:01:00+00:00,2021-12-06 20:11:00+00:00,2021-12-06 20:11:00+00:00,2021-12-06 20:11:00+00:00,American Airlines,AA,AAL,306.0,AA306,AAL306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93495,2022-02-28,landed,Greater Rochester International,America/New_York,ROC,KROC,B5,4.0,2022-02-28 13:00:00+00:00,2022-02-28 13:00:00+00:00,...,2022-02-28 14:29:00+00:00,2022-02-28 14:04:00+00:00,2022-02-28 14:04:00+00:00,2022-02-28 14:04:00+00:00,Air France,AF,AFR,2894.0,AF2894,AFR2894
93496,2022-02-28,landed,Greater Rochester International,America/New_York,ROC,KROC,B5,4.0,2022-02-28 13:00:00+00:00,2022-02-28 13:00:00+00:00,...,2022-02-28 14:29:00+00:00,2022-02-28 14:04:00+00:00,2022-02-28 14:04:00+00:00,2022-02-28 14:04:00+00:00,KLM,KL,KLM,5586.0,KL5586,KLM5586
93497,2022-02-28,landed,Greater Rochester International,America/New_York,ROC,KROC,B5,4.0,2022-02-28 13:00:00+00:00,2022-02-28 13:00:00+00:00,...,2022-02-28 14:29:00+00:00,2022-02-28 14:04:00+00:00,2022-02-28 14:04:00+00:00,2022-02-28 14:04:00+00:00,Virgin Atlantic,VS,VIR,1674.0,VS1674,VIR1674
93498,2022-02-28,landed,Greater Rochester International,America/New_York,ROC,KROC,B5,4.0,2022-02-28 13:00:00+00:00,2022-02-28 13:00:00+00:00,...,2022-02-28 14:29:00+00:00,2022-02-28 14:04:00+00:00,2022-02-28 14:04:00+00:00,2022-02-28 14:04:00+00:00,Delta Air Lines,DL,DAL,5076.0,DL5076,DAL5076


#### 4.  Fill up  arrival delay or departure delay based on the data

In [130]:
flights_df['departure_delay'] = flights_df['departure_delay'].fillna((flights_df['departure_actual'] - flights_df['departure_scheduled']).astype('timedelta64[m]')) 

flights_df['arrival_delay']=(flights_df['arrival_actual'] - flights_df['arrival_scheduled']).astype('timedelta64[m]') 
#if no arrival_actual, we assume that flights arrive on time
flights_df['arrival_delay'] = flights_df['arrival_delay'].fillna(0)

flights_df.to_csv('./data/winter_2021_clean.csv')

## Clean airlines_data

#### 1.remove columns that has more than 40% missing data, and keep useful columns

In [146]:
airlines_df = pd.read_csv('./data/airport.csv')
print('Before:', airlines_df.columns.tolist())
airlines_df = airlines_df.replace([0,' ','NULL'],np.nan)
airlines_df = airlines_df.dropna(axis=1,thresh=len(airlines_df)*0.60)
airlines_df.drop(['airport_id','iata_code', 'city_iata_code','timezone'], axis=1)
print('After:', airlines_df.columns.tolist())

Before: ['id', 'gmt', 'airport_id', 'iata_code', 'city_iata_code', 'icao_code', 'country_iso2', 'geoname_id', 'latitude', 'longitude', 'airport_name', 'country_name', 'phone_number', 'timezone']
After: ['id', 'gmt', 'airport_id', 'iata_code', 'city_iata_code', 'icao_code', 'country_iso2', 'geoname_id', 'latitude', 'longitude', 'airport_name', 'country_name', 'timezone']


#### 2.remove all missing value

In [147]:
print('before #airport:',len(airlines_df))
airlines_df.dropna(inplace=True)
print('after #airport:',len(airlines_df))
flights_df.to_csv('./airport_clean.csv')

before #airport: 6705
after #airport: 5893
